In [ ]:
from ptl import get
get.setup(debug=True, driver=False)

In [ ]:
get.setup(debug=True, driver=False)

In [ ]:
pages=[]
stack=[]
visited=[]
def analyze_page(url):
    global stack, visited, pages
    result={}
    soup=get.soup(url, get=True)
    visited.append(url)
    print('Progress:\t\t'+str(len(pages))+'/'+str(len(stack)))
    result['a']=[a for a in get.bes(div='a', parent=soup) if 'href' in str(a) and len(a['href'])> 3 and get.dmr(url) in a['href']]
    links=[a['href'] for a in result['a']]
    links=distinct(links)
    stack=distinct(stack+links)
    pages.append({'URL':url, 'SOUP':soup})
get.START_TIME=get.dtn()
print('Start time: \t'+get.START_TIME)
analyze_page('https://ukuleleunderground.com/')
for url in stack:
    analyze_page(url)
notvisited=list(set(stack)-set(stack).intersection(set(visited)))
while len(notvisited)!=0:
    for url in notvisited:
        #multi threadable
        analyze_page(url)
    notvisited=list(set(stack)-set(stack).intersection(set(visited)))
print('Start time: \t'+get.START_TIME+'\nFinish time: \t'+get.dtn())

In [ ]:
def distinct(alist):
    return list(dict.fromkeys(alist))
url='https://ukuleleunderground.com/'
VISISTED=[]
STACK=[url]
N_THREADS= 5
THRESHOLD= 3 
DOWNLOADED=[]
iteration=0
while len(STACK)!=0:
    get.log('> Iteration: '+str(iteration)+', stack size: '+str(len(STACK)))
    get.log('> Downloaded:'+str(len(DOWNLOADED)))
    stack=[{'URL':url} for url in STACK if '.pdf' not in url and '/members/' not in url]
    #print('> test'+str(get.split(STACK, N_THREADS)))
    if len(stack)/N_THREADS > THRESHOLD:
        N_THREADS=get.ceil(len(stack)/THRESHOLD)
    get.run_threads(get.split(stack, N_THREADS), get.soups, folder='nodriver')
    get.log('> Downloaded data')
    VISISTED=VISISTED+STACK
    new_links=[]
    for item in stack:
        if item['SOUP']:
            item['a']=[a for a in get.bes(None, div='a', parent=item['SOUP']) if 'href' in str(a)]
            links=[a['href'] for a in item['a'] if len(a['href'])> 3 and get.dmr(url) in a['href']]
            new_links=new_links+distinct(links)
    get.log('> New links: '+str(len(new_links)))
    DOWNLOADED=DOWNLOADED+stack[:500]
    STACK = list(set(new_links)-set(new_links).intersection(set(VISISTED)))[:1000]
    iteration+=1
    if iteration==2: break

In [ ]:
get.save(DOWNLOADED, get.dmn(url).upper()+'/'+get.START_TIME)

In [ ]:
req=get.make_request(STACK[0])

In [ ]:
df = get.DataFrame(DOWNLOADED)

In [ ]:
def pr(tag):
    _tag=get.bes(None, div=tag, parent=row['SOUP'])
    print(tag+':\t', len(_tag) if _tag else 0)
    for i in _tag:
        try:
            if tag=='meta':
                meta={}
                if i.has_attr('name'):    
                    meta[i['name']] = i['content']
                elif i.has_attr('charset'):
                    meta['charset'] = i['charset']
                print(meta)
            else:
                meta=i.text
            print(len(meta),'\t'+ str(meta))
        except: get.pe(str(i))

In [ ]:
for i in df.iloc[0]['SOUP'].findAll('img'):
    if i.has_attr('alt'):
        if len(i['alt'])>1:
            print('Image have alt text')
        else:
            print('Image have alt tag but no alt text:\n', i['src'])
    else:
        print('Image doesn\'t have a alt tag:', i['src'])

In [ ]:
def check_duplication(soup):
    
    df = get.df(soup.text.split('\n')).duplicated

In [ ]:
df.iloc[0]['SOUP'].findAll('p')

In [ ]:
def get_df(tag):
    text=[{'Text': row.text.strip(), 'Source':row} for row in df.iloc[0]['SOUP'].findAll(tag)]
    return get.DataFrame(text)

In [ ]:
get_df('video').groupby('Text').count() if len(get_df('video'))>0 else None

In [ ]:
get_df('p').groupby('Text').count()

In [ ]:
get_df('h1').groupby('Text').count()

In [ ]:
get_df('h2').groupby('Text').count()

In [ ]:
get_df('div').groupby('Text').count()

In [ ]:
for i, row in df.iterrows():
    print('URL:\t',row['URL'])
    pr('title')    
    pr('meta')    
    pr('h1')    
    pr('h2')   
    pr('video')    
    pr('video')    

In [ ]:
rq = get.get(url)


In [ ]:
soup = get.bs(rq.content, 'html.parser')
rs = [{'HEADERS':rq.headers,
       'CONTENT TYPE':rq.headers['Content-Type'],
       'CONTENT LENGTH':rq.headers['Content-Length'],
       'SERVER':rq.headers['Server'],
       'DATE':rq.headers['Date'],
      'ENCODE':rq.encoding,
      'COOKIES':rq.cookies,
      'RESPONSE TIME':rq.elapsed,
      'HISTORY':rq.history,
      'REDIRECT TYPE':'NONE' if not rq.is_redirect else 'PERMANENT' if rq.is_permanent_redirect else 'TEMPERARY',
      'REDIRECT':rq.is_redirect,
      'LINKS':rq.links,
      'NEXT LINK':rq.next,
      'STATUS':rq.reason,
       'STATUS CODE':rq.status_code,
      'FINAL REACH URL':rq.url,
     }]
checkfor('title', soup, rs[0])
checkfor('h1', soup, rs[0])
checkfor('h2', soup, rs[0])
checkfor('title', soup, rs[0])
checkfor('meta', soup, rs[0])
checkfor('link', soup, rs[0])
get.DataFrame(rs)

In [ ]:
def checkfor(tag, soup, rs):
    tags=soup.findAll(tag)
    for i, _tag in enumerate(tags):
        if tag=='meta':
            if _tag.has_attr('name'):    
                rs[_tag['name'].upper()] = _tag['content']
                rs[_tag['name'].upper()+' LENGTH'] = len(_tag['content'])
        elif tag=='link':
            if _tag.has_attr('rel'):    
                rs[_tag['rel'][0].upper()] = _tag['href']
                rs[_tag['rel'][0].upper()+' LENGTH'] = len(_tag['href'])
        else:
            rs[tag.upper()+' '+str(i)]=_tag.text
            rs[tag.upper()+' '+str(i)+' LENGTH']=len(_tag.text)

In [ ]:
get.save(get.DataFrame(rs), 'sf_test')

In [ ]:
get.soup('https://ukuleleunderground.com/page-sitemap.xml')

In [ ]:
urls=[a.text for a in get.bes(None, 'loc')]

In [ ]:
get.soup('https://ukuleleunderground.com/sitemap_index.xml')
locs=[loc.text for loc in get.bes(None, 'loc')]
urls=[]
for loc in locs:
    get.soup(loc)
    print(loc)
    urls = urls+ [{'URL':s.text, 'SOURCE':loc} for s in get.bes(None, 'loc')]
print(len(urls))

In [ ]:
len(get.DataFrame(urls)['URL'])

In [ ]:
def distinct(alist):
    return list(dict.fromkeys(alist))

In [ ]:
distinct(get.DataFrame(urls)['URL'])

In [ ]:
data= get.DataFrame(urls)
data.groupby('URL').count()

In [ ]:
data[data.duplicated(['SOURCE'])].groupby('SOURCE').count()

In [ ]:
data[data['URL']=='https://ukuleleunderground.com/courses/applied-music-theory/']

In [ ]:
grouped_df=data[data.duplicated(['URL'])].groupby('SOURCE')
for key, item in grouped_df:
    for v,i in item.iterrows():
        duplicate=data[data['URL']==i['URL']]
        print('DUPLICATE:',duplicate['URL'].iloc[0])
        print('SOURCE  0:',duplicate['SOURCE'].iloc[0])
        print('SOURCE  1:',duplicate['SOURCE'].iloc[1])

In [ ]:
len()

In [ ]:
from ptl import get
get.setup(debug=True, driver=False)

In [ ]:
import numpy
get.N_FAIL_REQUEST=1
def soups(rows, folder, t_id):
    for row in rows:
        try:    
            get.log('> GET '+row['URL'])
            rq = get.get(row['URL'])
            row['ADDRESS']=row['URL']
            row['HEADERS']=rq.headers
            row['CONTENT TYPE']=rq.headers['Content-Type'] if 'Content-Type' in rq.headers else ''
            #row['CONTENT LENGTH']=rq.headers['Content-Length'] if 'Content-Length' in rq.headers else ''
            #row['SERVER']=rq.headers['Server']
            row['DATE']=rq.headers['Date']
            #row['ENCODE']=rq.encoding
            row['RESPONSE TIME']=rq.elapsed
            row['REDIRECT TYPE']='NONE' if not rq.is_redirect else 'PERMANENT' if rq.is_permanent_redirect else 'TEMPERARY'
            row['REDIRECT']=rq.is_redirect
            row['STATUS']=rq.reason
            row['STATUS CODE']=rq.status_code
            row['FINAL REACH URL']=rq.url
            row['HTTP/HTTPS']='HTTPS' if 'https' in rq.url[:10] else 'HTTP'
            row['SOUP'] = get.bs(rq.content, 'html.parser')
            check('title', row)
            check('h1', row)
            check('meta', row)
            check('img', row)
            #check('h2', row)
            #check('title', row)
            #check('link', row)
        except: get.pe()
def get_status(_len):
    if _len>160: return "TOO LONG"
    if _len<50: return 'TOO SHORT' 
    return 'GOOD'
def check(tag, row):
    try:
        tags=row['SOUP'].findAll(tag)
        if tag=='img':
            row['Number of images'.upper()] = len(tags)
            imgs_with_tag=[img for img in tags if 'alt' in img]

            row['Number of with alt-tag'.upper()] = len(imgs_with_tag)
            row['Number of without alt-tag'.upper()] = len(tags) - len(imgs_with_tag)
            alt_tags=numpy.array([img['alt'] for img in imgs_with_tag])
            row['alt-tag Minimum length'.upper()] = alt_tags.min()
            row['alt-tag Average length'.upper()] = alt_tags.mean()
            row['alt-tag Maximum length'.upper()] = alt_tags.max()
            df = get.DataFrame(alt_tags)
            row['Number of Alt-Tags that are duplicated'.upper()] = len(df[df.duplicated])
            return
        for i, _tag in enumerate(tags):
            if tag=='meta':
                if _tag.has_attr('name') and (_tag['name'].lower()=='description' or
                                              _tag['name'].lower()=='keywords'):
                    name='META '+_tag['name']
                    row[name] = _tag['content']
                    _len = len(row[name])
                    row[name+' length'.upper()] = _len
                    row[name+' status'.upper()] = get_status(_len)

            elif tag=='link':
                if _tag.has_attr('rel') and _tag['rel'].lower()=='canonical': 
                    name=_tag['rel'].upper()
                    row[name.upper()] = _tag['href']
                    row[name.upper()+' length'.upper()] = len(_tag['href'])
            else:
                name=tag.upper()
                if 'h' in tag:
                    name=name+' '+str(i)
                row[name]=_tag.text
                _len = len(row[name])
                row[name+' length'.upper()]=_len
                row[name+' status'.upper()] = get_status(_len)
        if 'META DESCRIPTION' not in row.keys:
            row['META DESCRIPTION']= '[Missing meta description]'
    except: get.pe()
def distinct(alist):
    return list(dict.fromkeys(alist))
url='https://nativeenergy.com/'
try:
    robots = get.get(url+'/robots.txt')
    if not robots:
        get.log('> robots.txt not found')
except: 
    get.log('> robots.txt not found')
    get.pe()

VISISTED=[]
STACK=[url]
N_THREADS= 5
THRESHOLD= 3 
DOWNLOADED=[]
iteration=1
while len(STACK)!=0:
    get.log('> Iteration:  '+str(iteration)+', stack size: '+str(len(STACK)))
    get.log('> Downloaded: '+str(len(DOWNLOADED)))
    stack=[{'URL':url} for url in STACK if '.pdf' not in url and '/members/' not in url]
    #print('> test'+str(get.split(STACK, N_THREADS)))
    if len(stack)/N_THREADS > THRESHOLD:
        N_THREADS=get.ceil(len(stack)/THRESHOLD)
    get.log('> Getting content')
    soups(stack, '', '')
    #get.run_threads(get.split(stack, N_THREADS), soups, folder='nodriver')
    VISISTED=VISISTED+STACK
    new_links=[]
    for item in stack:
        if 'SOUP' in item and item['SOUP']:
            item['a']=[a for a in get.bes(None, div='a', pr=item['SOUP']) if 'href' in str(a)]
            links=[a['href'] for a in item['a'] if len(a['href'])> 3]
            links=[(url+a).replace('//','/').replace(':/', '://') for a in links 
                   if '//' not in a and 
                   '?' not in a and 
                   'javascript' not in a]
            links=distinct(links)
            new_links=new_links+links
    get.log('> New links:  '+str(len(new_links)))
    DOWNLOADED=DOWNLOADED+stack
    STACK = list(set(new_links)-set(new_links).intersection(set(VISISTED)))[:2]
    iteration+=1
    if iteration==3:
        break

In [ ]:
_len = 50

In [ ]:
from unidecode import unidecode
def FormatString(s):
    if isinstance(s, unicode):
        try:
            s.encode('ascii')
            return s
        except:
            return unidecode(s)
    else:
        return s

#df2 = df1.applymap(FormatString) 


In [ ]:
grouped_df=df[df.duplicated(['TITLE 0'])].groupby('TITLE 0')
for key, item in grouped_df:
    for v,i in item.iterrows():
        duplicate=df[df['TITLE 0']==i['TITLE 0']]
        print('DUPLICATE:'+str(duplicate))

In [ ]:
df[df.duplicated(['TITLE 0'])].groupby('URL').count()

In [ ]:
df = get.DataFrame(DOWNLOADED).applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df = df.sort_values(['DESCRIPTION LENGTH', 'TITLE 0 LENGTH'], ascending=False)
get.save(df, 'SFA/test', ['SOUP'])

In [ ]:
import pandas as pd

writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()


In [ ]:
get.isdir('SFA')
df = df.sort_values(['DESCRIPTION LENGTH', 'TITLE 0 LENGTH'], ascending=False)
df.to_csv('SFA/test.csv', encoding='utf8', columns=df.columns.drop(['SOUP']))

In [ ]:
for item in stack:
    if 'SOUP' in item and item['SOUP']:
        item['a']=[a for a in get.bes(None, div='a', pr=item['SOUP']) if 'href' in str(a)]
        links=[a['href'] for a in item['a'] if len(a['href'])> 3]
        links=[(url+a).replace('//','/').replace(':/', '://') for a in links 
               if '//' not in a and 
               '?' not in a and 
               'javascript' not in a]
        links=distinct(links)
        new_links=new_links+links

In [ ]:
get.get('https://nativeenergy.comassets/files/Light_Boxes/PSP_CurtisPackaging.jpg')

In [ ]:
item['a']=[a for a in get.bes(None, div='a', pr=item['SOUP']) if 'href' in str(a)]
links=[root+a['href'] for a in item['a'] if len(a['href'])> 3 and get.dmr(url) in a['href']]
links

In [ ]:
root=get.dmr(url)
root

In [ ]:
links=[a['href'] for a in item['a'] if len(a['href'])> 3]
links=distinct([root+a for a in links if '/' in a and '//' not in a])

In [ ]:
[a['href'] for a in get.bes(None, 'a') if a]

In [ ]:
get.bes(None, 'a')

In [ ]:
robots=None
def request(url):
    global robots
    if not robots:
        robots=get.dmr(url)+'/robots.txt'
        robots =get.get(robots).content
    #print('> GET:', url)
    rq = get.get(url)
    soup = get.bs(rq.content, 'html.parser')
    rs = {'HEADERS':rq.headers,
       'CONTENT TYPE':rq.headers['Content-Type'] if 'Content-Type' in rq.headers else '',
       'CONTENT LENGTH':rq.headers['Content-Length'] if 'Content-Length' in rq.headers else '',
       'SERVER':rq.headers['Server'],
       'DATE':rq.headers['Date'],
      'ENCODE':rq.encoding,
      'COOKIES':rq.cookies,
      'RESPONSE TIME':rq.elapsed,
      'HISTORY':rq.history,
      'REDIRECT TYPE':'NONE' if not rq.is_redirect else 'PERMANENT' if rq.is_permanent_redirect else 'TEMPERARY',
      'REDIRECT':rq.is_redirect,
      'LINKS':rq.links,
      'NEXT LINK':rq.next,
      'STATUS':rq.reason,
       'STATUS CODE':rq.status_code,
      'FINAL REACH URL':rq.url,
           'HTTP/HTTPS': 'HTTPS' if 'https' in rq.url[:10] else 'HTTP',
           'ROBOTS.TXT':robots,
         }
    checkfor('title', soup, rs)
    checkfor('h1', soup, rs)
    checkfor('h2', soup, rs)
    checkfor('title', soup, rs)
    checkfor('meta', soup, rs)
    checkfor('link', soup, rs)
    return rs
rows=[]
total=len(data['URL'])
for i, url in enumerate(data['URL']):
    print(i,'/',total,'>>>',url)
    rows.append(request(url))
    

In [ ]:
data['URL']

In [ ]:
rq = get.get('https://ukuleleunderground.com/course-level/beginner/')

In [ ]:
get.save(get.DataFrame(rows), 'test_1')

In [ ]:
get.DataFrame(rows)

In [ ]:
rq.headers

In [ ]:
#The apparent encoding, provided by the chardet library.
rq.apparent_encoding

In [ ]:
#Encoding to decode with when accessing r.text.
rq.encoding

In [ ]:
#A CookieJar of Cookies the server sent back.
rq.cookies

In [ ]:
#Response status code
rq

In [ ]:
rq.status_code 

In [ ]:
#The amount of time elapsed between sending the request and
#the arrival of the response (as a timedelta). 
#This property specifically measures the time taken between 
#sending the first byte of the request and finishing parsing the headers. 
#It is therefore unaffected by consuming the response content or the 
#value of the stream keyword argument.
rq.elapsed

In [ ]:
#A list of Response objects from the history of the Request.
#Any redirect responses will end up here. 
#The list is sorted from the oldest to the most recent request.
rq.history 

In [ ]:
#True if this Response one of the permanent versions of redirect.
rq.is_permanent_redirect

In [ ]:
#True if this Response is a well-formed HTTP redirect 
#that could have been processed automatically 
rq.is_redirect

In [ ]:
#Returns the parsed header links of the response, if any.
rq.links

In [ ]:
#Returns a PreparedRequest for the next request 
#in a redirect chain, if there is one.
rq.next

In [ ]:
#Returns True if status_code is less than 400.
rq.ok

In [ ]:
rq.reason 

In [ ]:
#Final URL location of Response. In case got redirected.
rq.url